## Measuring energy and energy variance with classical shadows

Related Paper [Huang et al, Nat Phys 2020](https://doi.org/10.1038/s41567-020-0932-7)

In [3]:
using ITensors,ITensorMPS
using ProgressBars
using RandomMeas
using Statistics

# Ising Hamiltonian

We first construct the Hamiltonian of the transverse field Ising model with N qubits.

In [7]:
N = 8 # Number of qubits
ξ = siteinds("Qubit", N) # ITensor site indices for classical shadows
# Define the Hamiltonian
J = -1. # Ising interaction strength
B = 1. # Transverse field strength
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising XX interactions
  ampo .+= J , "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field Z
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ) # Hamiltonian as MPO
H2 = apply(H,H); # H^2 for later use

# Groundstate energy and variance

First, we compute the ground state $\psi_G$ of the transverse field Ising model with DMRG.

In [5]:
# Define the DMRG parameters 
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)

# Run DMRG to get the ground state
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψG = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-9.735866623782888  maxlinkdim=4 maxerr=1.55E-16 time=6.045
After sweep 2 energy=-9.837178296476658  maxlinkdim=13 maxerr=9.69E-12 time=0.005
After sweep 3 energy=-9.837951340730063  maxlinkdim=9 maxerr=7.08E-11 time=0.005
After sweep 4 energy=-9.837951446192639  maxlinkdim=6 maxerr=7.54E-11 time=0.013
After sweep 5 energy=-9.83795144619905  maxlinkdim=6 maxerr=6.91E-11 time=0.005

Ground state energy:  -9.83795144619905

---------------------------------------



We compute its energy and variance.

In [6]:
E_G = real(inner(ψG', H,ψG))
println("final energy ",E_G)
Var_E_G = real(inner(ψG', H2,ψG))-E_G^2
println("final energy variance ",Var_E_G)

final energy -9.83795144619906
final energy variance 1.2634600921046513e-8


We estimate its energy and variance with classical shadows

In [6]:
NU = 1000 # Number of random unitaries
NM = 400 # Number of measurements per unitary 

1000

In [7]:
Es = Float32[]
E2s = Float32[]
for r in ProgressBar(1:NU, printing_delay=5) # Loop over random unitaries
        u = get_rotations(ξ,"Haar") # Local random unitary
        data = get_RandomMeas_MPS(ψG,u,NM) # Perform simulated randomized measurements
        for m in 1:NM
            ρ = get_shadow_factorized(data[m,:],ξ,u) # Get the classical shadow
            push!(Es,get_expect_shadow(H,ρ,ξ)) # Estimate the expectation value of the Hamiltonian
            push!(E2s,get_expect_shadow(H2,ρ,ξ)) # Estimate the expectation value of the Hamiltonian squared
        end
end

0.0%┣                                             ┫ 0/400 [00:01<-8:-46, -1s/it]
0.2%┣                                          ┫ 1/400 [00:10<Inf:Inf, InfGs/it]
5.5%┣██▌                                           ┫ 22/400 [00:15<04:25, 1it/s]
10.5%┣████▊                                        ┫ 42/400 [00:20<02:53, 2it/s]
15.2%┣██████▉                                      ┫ 61/400 [00:25<02:21, 2it/s]
19.8%┣████████▉                                    ┫ 79/400 [00:30<02:03, 3it/s]
24.5%┣███████████                                  ┫ 98/400 [00:35<01:50, 3it/s]
29.5%┣█████████████                               ┫ 118/400 [00:40<01:37, 3it/s]
34.5%┣███████████████▏                            ┫ 138/400 [00:46<01:27, 3it/s]
39.5%┣█████████████████▍                          ┫ 158/400 [00:51<01:18, 3it/s]
44.5%┣███████████████████▋                        ┫ 178/400 [00:56<01:10, 3it/s]
49.5%┣█████████████████████▉                      ┫ 198/400 [01:01<01:03, 3it/s]
54.5%┣██████████████████████

In [8]:
Es_m = mean(Es)
Es_ste = std(Es)/sqrt(NU*NM)
E2s_m = mean(E2s)
E2s_ste = std(E2s)/sqrt(NU*NM)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy of the circuit generated state -9.82±0.01
Estimated expectation value of H^2 of the circuit generated state 96.3±0.3
Estimated energy variance of the circuit generated state -0.2±-0.08


# Output state of quantum circuit

We construct a state as output of a quantum circuit.

In [9]:
η = 10 # Number of layer blocks
m = 4 # Number of layers per block
# Build the gate structure
circuit = ITensor[]
for d in 1:η
    xx_layer = [op("Rxx", ξ[j], ξ[j+1], ϕ=1*d/η/m) for j in 1:(N - 1)] # Rxx gates TODO: PastaQ dependence removed
    z_layer = [op("Rz", ξ[j], θ=2*B/m) for j in 1:N] # Rxx gates TODO: PastaQ dependence removed
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);
ψt = apply(circuit,ψ0; cutoff=1E-8)
normalize!(ψt) # Normalize the state TODO: Is this ok?
#orthogonalize!(ψt,1) TODO: why do we need this? 
#ψt[1] /= norm(ψt[1]) TODO: why do we need this?


MPS
[1] ((dim=2|id=114|"Qubit,Site,n=1"), (dim=2|id=62|"Link,n=1"))
[2] ((dim=2|id=610|"Qubit,Site,n=2"), (dim=2|id=62|"Link,n=1"), (dim=4|id=219|"Link,n=1"))
[3] ((dim=2|id=679|"Qubit,Site,n=3"), (dim=4|id=219|"Link,n=1"), (dim=8|id=626|"Link,n=1"))
[4] ((dim=2|id=294|"Qubit,Site,n=4"), (dim=8|id=626|"Link,n=1"), (dim=13|id=847|"Link,n=1"))
[5] ((dim=2|id=483|"Qubit,Site,n=5"), (dim=13|id=847|"Link,n=1"), (dim=8|id=133|"Link,n=1"))
[6] ((dim=2|id=907|"Qubit,Site,n=6"), (dim=8|id=133|"Link,n=1"), (dim=4|id=447|"Link,n=1"))
[7] ((dim=2|id=198|"Qubit,Site,n=7"), (dim=4|id=447|"Link,n=1"), (dim=2|id=351|"Link,n=1"))
[8] ((dim=2|id=399|"Qubit,Site,n=8"), (dim=2|id=351|"Link,n=1"))


We compute its energy and variance.

In [10]:
E_th = real(inner(ψt', H,ψt)) # Energy of ψt TODO: I don't understand the naming convention here. What does the _th stand for? Thermal?
println("final energy ",E_th)
Var_E_th = real(inner(ψt', H2,ψt))-E_th^2 # Energy variance of ψt
println("final energy variance ",Var_E_th)

final energy -9.47182617618307
final energy variance 1.6950453000525556


In [21]:
NU = 1000 # Number of random unitaries
NM = 400 # Number of measurements per unitary 

400

In [22]:
Es = Float32[]
E2s = Float32[]
for r in ProgressBar(1:NU, printing_delay=5) # Loop over random unitaries
        u = get_rotations(ξ,"Haar") # Local random unitary
        data = get_RandomMeas_MPS(ψt,u,NM) # Perform simulated randomized measurements
        for m in 1:NM
            ρ = get_shadow_factorized(data[m,:],ξ,u) # Get the classical shadow
            push!(Es,get_expect_shadow(H,ρ,ξ)) # Estimate the expectation value of the Hamiltonian
            push!(E2s,get_expect_shadow(H2,ρ,ξ)) # Estimate the expectation value of the Hamiltonian squared
        end
end

0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]
4.7%┣██▏                                          ┫ 47/1.0k [00:05<01:45, 9it/s]
9.6%┣████▎                                        ┫ 96/1.0k [00:10<01:37, 9it/s]


LoadError: InterruptException:

In [20]:
Es_m = mean(Es)
Es_ste = std(Es)/sqrt(NU*NM)
E2s_m = mean(E2s)
E2s_ste = std(E2s)/sqrt(NU*NM)
Evar = E2s_m - Es_m^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy of the circuit generated state -9.46±0.03
Estimated expectation value of H^2 of the circuit generated state 90.9±0.5
Estimated energy variance of the circuit generated state 1.3±1.0
